## 红酒质量数据回归探索

---

红酒质量数据集同样来自于 UCI 数据集网站。首先，挑战导入所需模块。

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

读取并预览数据集，同时查看数据集列属性。

In [2]:
data = pd.read_csv(
    'https://labfile.oss.aliyuncs.com/courses/1283/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


下面，将数据集按 7:3 分割成训练集和测试集，设置 `random_state=17`，同时使用 `StandardScaler` 对特征数据规范化。

In [5]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3,
                                                          random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

### 线性回归

使用普通最小二乘法线性回归训练一个回归预测模型。

In [6]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

<i class="fa fa-question-circle" aria-hidden="true"> 问题：</i>训练数据和测试数据上的平均绝对误差 MSE 值是多少？

In [7]:
print("Mean squared error (train): %.3f" %
      mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" %
      mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


接下来，你需要将回归系数按绝对值从大到小进行排序。值得注意的是，特征所对应的拟合系数绝对值越大，即代表该特征对目标值的影响越大。这里建议你使用 `pandas.DataFrame` 对数据进行展示。

<i class="fa fa-question-circle" aria-hidden="true"> 问题：</i>该线性回归模型中，哪些特征对目标值的影响更大？

In [8]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'coef_abs': np.abs(linreg.coef_)},
                           index=data.columns.drop('quality'))
linreg_coef.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


### Lasso 回归

使用规范化后的数据训练一下 Lasso 回归模型，设置 $\alpha = 0.01$，且 `random_state=17`。

In [9]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

<i class="fa fa-question-circle" aria-hidden="true"> 问题：</i> 该 Lasso 回归模型中，哪些特征对目标值的影响最小？

In [10]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'coef_abs': np.abs(lasso1.coef_)},
                           index=data.columns.drop('quality'))
lasso1_coef.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


接下来，使用 LassoCV 来寻找最合适的 $\alpha$ 参数。

In [11]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [12]:
lasso_cv.alpha_

0.0002833096101839324

<i class="fa fa-question-circle" aria-hidden="true"> 问题：</i> 上面调参得到的 Lasso 回归模型中，哪些特征对目标值的影响最小？

In [13]:
lasso_cv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'coef_abs': np.abs(lasso_cv.coef_)},
                             index=data.columns.drop('quality'))
lasso_cv_coef.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
density,-0.648161,0.648161
residual sugar,0.526883,0.526883
volatile acidity,-0.192049,0.192049
pH,0.146549,0.146549
alcohol,0.137115,0.137115
fixed acidity,0.093295,0.093295
sulphates,0.060939,0.060939
free sulfur dioxide,0.042698,0.042698
total sulfur dioxide,0.012969,0.012969
chlorides,0.006933,0.006933


<i class="fa fa-question-circle" aria-hidden="true"> 问题：</i> 调参得到的 Lasso 回归模型中，训练数据和测试数据上的平均绝对误差 MSE 值是多少？

In [14]:
print("Mean squared error (train): %.3f" %
      mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" %
      mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


### 随机森林

使用默认参数训练一个随机森林回归模型，设置 `random_state=17`。

In [15]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

和上面的过程相似，接下来请自行对随机森林回归模型进行调餐，并探索不同特征对目标值的影响程度，最终得到 MSE 值。

In [16]:
print("Mean squared error (train): %.3f" %
      mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" %
      np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train,
                                     scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" %
      mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.075
Mean squared error (cv): 0.460
Mean squared error (test): 0.422


In [17]:
forest_params = {'max_depth': list(range(18, 20)),
                 'max_features': list(range(6, 8))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17),
                                   forest_params,
                                   scoring='neg_mean_squared_error',
                                   n_jobs=-1, cv=5,
                                   verbose=True)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    4.4s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [18, 19], 'max_features': [6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [18]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 19, 'max_features': 7}, -0.4346519750542217)

In [19]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_,
                                                                       X_train_scaled, y_train,
                                                                       scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout,
                                                             locally_best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.457
Mean squared error (test): 0.410


In [20]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_,
                             columns=['coef'], index=data.columns[:-1])
rf_importance.sort_values(by='coef', ascending=False)

,coef
alcohol,0.224432
volatile acidity,0.119393
free sulfur dioxide,0.116147
pH,0.072806
total sulfur dioxide,0.071318
residual sugar,0.070160
density,0.069367
chlorides,0.067982
fixed acidity,0.064268
citric acid,0.062945


最终，你应该可以发现葡萄酒质量数据集中特征和目标之间呈现较强的非线性关系，所以随机森林在这项任务中工作得更好。

---

<div style="background-color: #e6e6e6; margin-bottom: 10px; padding: 1%; border: 1px solid #ccc; border-radius: 6px;text-align: center;"><a href="https://nbviewer.jupyter.org/github/shiyanlou/mlcourse-answers/tree/master/" title="挑战参考答案"><i class="fa fa-file-code-o" aria-hidden="true"> 查看挑战参考答案</i></a></div>